In [3]:
from functools import lru_cache
from typing import List

def custo_imediato(estoque: int, q: int, dia: int, consumo: List[int], c_estoque: int, c_pedido: int, c_falta: int) -> int:
    falta = max(0, consumo[dia] - (estoque + q))
    custo = estoque * c_estoque
    if q > 0:
        custo += c_pedido
    custo += falta * c_falta
    return custo

def dp_rec_puro(dia: int, estoque: int, consumo: List[int], max_cap: int, c_estoque: int, c_pedido: int, c_falta: int) -> int:
    if dia == len(consumo):
        return 0
    best = float('inf')
    # decisão: q de 0 até caber no estoque
    for q in range(0, max_cap - estoque + 1):
        estp = min(max_cap, max(0, estoque + q - consumo[dia]))
        best = min(best, custo_imediato(estoque, q, dia, consumo, c_estoque, c_pedido, c_falta) +
                   dp_rec_puro(dia+1, estp, consumo, max_cap, c_estoque, c_pedido, c_falta))
    return int(best)

def make_dp_rec_memo(consumo: List[int], max_cap: int, c_estoque: int, c_pedido: int, c_falta: int):
    @lru_cache(maxsize=None)
    def _dp(dia: int, estoque: int) -> int:
        if dia == len(consumo):
            return 0
        best = float('inf')
        for q in range(0, max_cap - estoque + 1):
            estp = min(max_cap, max(0, estoque + q - consumo[dia]))
            best = min(best, custo_imediato(estoque, q, dia, consumo, c_estoque, c_pedido, c_falta) +
                       _dp(dia+1, estp))
        return int(best)
    return _dp

def dp_bottom_up(consumo: List[int], estoque_ini: int, max_cap: int, c_estoque: int, c_pedido: int, c_falta: int) -> int:
    n = len(consumo)
    dp = [[0]*(max_cap+1) for _ in range(n+1)]
    for d in range(n-1, -1, -1):
        for e in range(0, max_cap+1):
            best = float('inf')
            for q in range(0, max_cap - e + 1):
                estp = min(max_cap, max(0, e + q - consumo[d]))
                best = min(best, custo_imediato(e, q, d, consumo, c_estoque, c_pedido, c_falta) + dp[d+1][estp])
            dp[d][e] = int(best)
    return dp[0][estoque_ini]

def exemplo():
    # Exemplo didático que deve constar no relatório
    consumo     = [5, 4, 6, 3, 7]  # demanda/consumo por dia
    max_cap     = 15               # capacidade máxima de estoque
    c_estoque   = 1                # custo por unidade mantida em estoque ao final do dia
    c_pedido    = 5                # custo fixo por efetuar pedido (se q>0)
    c_falta     = 10               # penalidade por unidade em falta no dia
    estoque_ini = 10               # estoque inicial

    # 1) recursiva pura
    res_puro = dp_rec_puro(0, estoque_ini, consumo, max_cap, c_estoque, c_pedido, c_falta)

    # 2) recursiva com memoização
    dp_memo = make_dp_rec_memo(tuple(consumo), max_cap, c_estoque, c_pedido, c_falta)  # consumo como tupla p/ cache
    res_memo = dp_memo(0, estoque_ini)

    # 3) bottom-up
    res_bu = dp_bottom_up(consumo, estoque_ini, max_cap, c_estoque, c_pedido, c_falta)

    print("Resultados equivalentes das três abordagens:")
    print("Recursiva pura :", res_puro)
    print("Recursiva memo :", res_memo)
    print("Bottom-up      :", res_bu)

    assert res_puro == res_memo == res_bu, "As três abordagens devem produzir o mesmo custo ótimo."

if __name__ == "__main__":
    exemplo()

Resultados equivalentes das três abordagens:
Recursiva pura : 29
Recursiva memo : 29
Bottom-up      : 29
